In [1]:
import os
import sys

In [2]:
import pandas as pd
import glob
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline 


# Seaborn visualization library
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier

import scipy

import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics

import statsmodels.api as sm
from scipy import stats

import lxml
from io import StringIO 

In [3]:
pd.set_option('display.max_columns', 1000)
pd.options.display.max_rows=1000

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

plt.rcParams.update({'figure.max_open_warning': 0})


In [4]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# import scripts.supervised_prep
from scripts.supervised_prep import test_print
from scripts.supervised_prep import dataprep_X
from scripts.supervised_prep import dataprep_Y
from scripts.supervised_prep import results_summary_to_dataframe

In [5]:
test_print(2)

1


# Load Data

In [6]:
inroot = r"../data/input/07 Samsung UX Index - Web App Implementation/"
fname_data = inroot + r"Samsung UX Index Survey_Data.csv"
df_data = pd.read_csv(fname_data)
fname_vaxmap = inroot + r"Samsung UX Index Survey_Datamap.xlsx"
df_varmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=0)
df_valmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=1)



/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1218) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
df_addmap1 = pd.read_csv(inroot+"../additional_mapping/qxbrandxtrustxdrivers_quaitydrivers.csv")
df_addmap1.head()

,qxbrandxtrustxdrivers,Tag,quality_indicator,qxoverallxqualityxindicators,driver_category
0,qxbrandxtrustxdrivers_1,Transparent,Trust,qxoverallxqualityxindicators_1,Brand-Specific
1,qxbrandxtrustxdrivers_2,Ethical,Trust,qxoverallxqualityxindicators_1,Brand-Specific
2,qxbrandxtrustxdrivers_3,Respectful,Trust,qxoverallxqualityxindicators_1,Brand-Specific
3,qxbrandxtrustxdrivers_4,Integrity / honest,Trust,qxoverallxqualityxindicators_1,Brand-Specific
4,qxdrivers_1,Safe / secure,Trust,qxoverallxqualityxindicators_1,Core UX Principles


In [43]:
df_varmap[df_varmap.Variable.str.contains('d7_4')]

,Variable,Label,Short Label,Type,Minimum,Maximum,Description
1223,d7_4,D7. Asian or Asian American - What is your race?,NaN,integer,0,1,NaN


In [84]:
# SuperLabels = ['drivers', 'brandlovetrust', 'loyaltymetrics', 'overallquality',
#           'usagemetrics', 'activitiesximportance','activitiexsquality',
#           'activitiesxsatisfaction','activitiesxdrivers',
#            'upgradetrans','ECGexpect']

# SuperLabels = [x+"xallATTR" for x in SuperLabels]

# SuperList = [['qxdrivers_'+str(i+1) for i in range(34)],
#              [ x for x in df_data.columns if 'qxbrandx' in x ],
#             ['qxadvocacy01_1',    'qxadvocacy02_1',    'qxretention_1',    'qxenrichment_1'],
#             ['qxoverallxqualityxindicators_'+str(i+1) for i in range(4)],
#             [ x for x in df_data.columns if 'qxactivitiesximportance' in x ],
#             [ x for x in df_data.columns if 'qxactivitiesxqualityxindicators' in x ],
#             [ x for x in df_data.columns if 'qxactivitiesxsatisfaction' in x ],
#             [ x for x in df_data.columns if 'qxactivitiesxdrivers' in x ] ,
#          ['qxupgrade01_1', 'qxupgrade01_2', 'qxupgrade01_3', 'qxupgrade01_4', 
#           'qxupgrade01_5', 'qxupgrade01_6', 'qxupgrade01_7', 'qxupgrade01_8', 
#           'qxtransition_1'],
# [ 'att01_1','att01_2','att01_3','att02_1','att02_2','soc03','soc04_1',
#           'soc04_2','soc04_3','ret06_1','qxexpectations' ]
#             ]

# varlist_Z  = ['d1',    'd3_1',    'd3_2',    'd3_3',    'd3_4',    'd4', 'd6',    'd7_1',    
#             'd7_2',    'd7_3',    'd7_4',    'd7_5',    'd7_97',    'd7_99','qxcurrentxos',
#               'qxactivitiesxrecency_1', 'qxactivitiesxrecency_2', 'qxactivitiesxrecency_3', 
#               'qxactivitiesxrecency_4', 'qxactivitiesxrecency_5', 'qxactivitiesxrecency_6', 
#               'qxactivitiesxrecency_7', 'qxactivitiesxrecency_8', 'qxactivitiesxrecency_9', 
#               'qxactivitiesxrecency_10', 'qxactivitiesxrecency_11', 'qxactivitiesxrecency_12', 
#               'qxactivitiesxrecency_13', 'qxactivitiesxrecency_14', 'qxactivitiesxrecency_15', 
#               'qxactivitiesxrecency_16', 'qxactivitiesxrecency_17', 'qxactivitiesxrecency_18', 
#               'qxactivitiesxrecency_19', 'qxactivitiesxrecency_20', 'qxactivitiesxrecency_21', 
#               'qxactivitiesxrecency_22', 'qxactivitiesxrecency_23', 'qxactivitiesxrecency_24'
#               ]

ohdict = {'qxcurrentxos' :  None,
              'd4' :  None, 
              'd1' : None, 
              'hidagemodels' : None,
            'qxcurrentxmodel': None,
             'hbrand' : None,
             'hmodelquota' : None,
             'hmodelquota_reordered' : None,
             'Empowered_Customer_Groups' : None}
                 

var_y = 'att01_1'
# varlist_X  = ['d1',    'd3_1', 'd3_2',    'd3_3',    'd3_4',    'd4', 'd6',    'd7_1',    
#             'd7_2',    'd7_3',    'd7_4',    'd7_5',    'd7_97',    'd7_99', ]
varlist_X = ['qxactivitiesxrecency_1', 'qxactivitiesxrecency_2', 'qxactivitiesxrecency_3', 
              'qxactivitiesxrecency_4', 'qxactivitiesxrecency_5', 'qxactivitiesxrecency_6', 
              'qxactivitiesxrecency_7', 'qxactivitiesxrecency_8', 'qxactivitiesxrecency_9', 
              'qxactivitiesxrecency_10', 'qxactivitiesxrecency_11', 'qxactivitiesxrecency_12', 
              'qxactivitiesxrecency_13', 'qxactivitiesxrecency_14', 'qxactivitiesxrecency_15', 
              'qxactivitiesxrecency_16', 'qxactivitiesxrecency_17', 'qxactivitiesxrecency_18', 
              'qxactivitiesxrecency_19', 'qxactivitiesxrecency_20', 'qxactivitiesxrecency_21', 
              'qxactivitiesxrecency_22', 'qxactivitiesxrecency_23', 'qxactivitiesxrecency_24'
              ]

# Prep Data

In [85]:
# X = dataprep_X (df_data, varlist=varlist, bystring='elbow')

X, df_varmap = dataprep_X (df_data, df_varmap, df_valmap, varlist_X, ohdict=ohdict)

Y = dataprep_Y (df_data, var_y)


In [86]:
# X = X.values#.reshape(-1,1)
# Y = Y.values#.reshape(-1,1)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Linear Regression

In [88]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [89]:
X.columns

Index(['qxactivitiesxrecency_1', 'qxactivitiesxrecency_2', 'qxactivitiesxrecency_3', 'qxactivitiesxrecency_4', 'qxactivitiesxrecency_5', 'qxactivitiesxrecency_6', 'qxactivitiesxrecency_7', 'qxactivitiesxrecency_8', 'qxactivitiesxrecency_9', 'qxactivitiesxrecency_10', 'qxactivitiesxrecency_11', 'qxactivitiesxrecency_12', 'qxactivitiesxrecency_13', 'qxactivitiesxrecency_14', 'qxactivitiesxrecency_15', 'qxactivitiesxrecency_16', 'qxactivitiesxrecency_17', 'qxactivitiesxrecency_18', 'qxactivitiesxrecency_19', 'qxactivitiesxrecency_20', 'qxactivitiesxrecency_21', 'qxactivitiesxrecency_22', 'qxactivitiesxrecency_23', 'qxactivitiesxrecency_24'], dtype='object')

In [90]:
X_train2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train2)
est2 = est.fit()


df_est = results_summary_to_dataframe(est2)
df_est.head()

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,coeff,pvals,tvals,conf_lower,conf_higher
const,0.616510,2.190591e-22,9.823592,0.493448,0.739571
qxactivitiesxrecency_1,-0.062958,2.384081e-04,-3.679575,-0.096509,-0.029407
qxactivitiesxrecency_2,0.094313,2.306757e-06,4.735212,0.055258,0.133369
qxactivitiesxrecency_3,-0.049300,1.332428e-06,-4.846307,-0.069248,-0.029353
qxactivitiesxrecency_4,-0.005865,6.281431e-01,-0.484400,-0.029606,0.017877


In [91]:
df_est[(df_est.pvals<.05)].sort_values('coeff').round(5)

,coeff,pvals,tvals,conf_lower,conf_higher
qxactivitiesxrecency_16,-0.08315,0.00000,-5.07591,-0.11527,-0.05103
qxactivitiesxrecency_1,-0.06296,0.00024,-3.67957,-0.09651,-0.02941
qxactivitiesxrecency_9,-0.05315,0.00059,-3.43830,-0.08347,-0.02284
qxactivitiesxrecency_6,-0.05117,0.00000,-6.22805,-0.06728,-0.03506
qxactivitiesxrecency_3,-0.04930,0.00000,-4.84631,-0.06925,-0.02935
qxactivitiesxrecency_23,-0.04723,0.00000,-5.37781,-0.06445,-0.03001
qxactivitiesxrecency_17,-0.03752,0.03077,-2.16120,-0.07156,-0.00348
qxactivitiesxrecency_22,0.02913,0.00238,3.04119,0.01035,0.04791
qxactivitiesxrecency_19,0.06202,0.00003,4.14552,0.03268,0.09135
qxactivitiesxrecency_2,0.09431,0.00000,4.73521,0.05526,0.13337


In [71]:
# coeff_df = pd.DataFrame(regressor.coef_[0], varlist_X, columns=['Coefficient'])  

In [72]:
# y_pred = regressor.predict(X_test)

In [73]:
# y_test_list = [item for sublist in y_test for item in sublist]
# len(y_test_list)

In [74]:
# y_pred_list = [item for sublist in y_pred for item in sublist]
# len(y_pred_list)

In [75]:
# df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
# df1 = df.head(25)

In [76]:
# df1.plot(kind='bar',figsize=(10,8))
# plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
# plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# plt.show()


In [77]:
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [78]:
# r_sq = regressor.score(X, Y)
# print('coefficient of determination:', r_sq)

In [79]:
print('intercept:', regressor.intercept_)
print('slope:', regressor.coef_)

intercept: [0.38876612]
slope: [[ 0.0626472  -0.24876733 -0.51585356 -0.44991679  0.00268308 -0.31046024
  -0.12877546 -0.13586348 -0.22996411 -0.00843573 -0.40460502 -0.37156784
  -0.0110592  -0.00476251 -0.06425489 -0.05993279  0.13134915  0.04705293
  -0.03839268 -0.11004785  0.32888536  0.01767846 -0.23651597]]


# Poly regression

In [80]:
# PolynomialFeatures (prepreprocessing)
poly = PolynomialFeatures(degree=2)
X_ = poly.fit_transform(X)
X_train_ = poly.fit_transform(X_train)
X_test_ = poly.fit_transform(X_test)

In [81]:
# Instantiate
lg = LinearRegression()

# Fit
lg.fit(X_train_, y_train)

# Obtain coefficients
# lg.coef_

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [82]:
X_train_2 = sm.add_constant(X_train_)
est_poly = sm.OLS(y_train, X_train_2)
est_poly2 = est_poly.fit()
df_est_poly2 = results_summary_to_dataframe(est_poly2)
df_est_poly2.index = poly.get_feature_names(X.columns)


/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [83]:
df_est_poly2[(df_est_poly2.pvals<.05)].sort_values('coeff')

,coeff,pvals,tvals,conf_lower,conf_higher
d4_1 d1_2,-0.568891,0.023366,-2.268823,-1.060582,-0.077200
d6^2,-0.002808,0.000217,-3.704474,-0.004295,-0.001322
d6 d7_1,0.344772,0.049093,1.968769,0.001371,0.688173
d6 d7_99,0.347531,0.048040,1.978010,0.002999,0.692064
d6 d7_4,0.356518,0.043328,2.021592,0.010697,0.702339


In [ ]:
df_est_poly2[(df_est_poly2.index.str.contains('d1_2'))&(df_est_poly2.pvals<.05)]

# Decision Trees

In [32]:
from sklearn import tree

clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train,y_train)

In [33]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file='../data/output/tree_unstd.dot',
                                feature_names=X_train.columns,
                               filled=True, rounded= True, max_depth = 5) 

In [34]:
!dot -Tpng ../data/output/tree_unstd.dot -o ../data/output/tree_unstd.png

In [ ]:
![title]("../data/output/tree.png")